In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/hubble/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/hubble/')
import pandas as pd

os.environ['PATH'] = '/Users/ravi.tej/anaconda3/envs/bertopicenv/bin:/Users/ravi.tej/anaconda3/condabin:/usr/bin:/bin:/usr/sbin:/sbin'

In [2]:
from src.RAGTraining.S3DatasetService import S3DatasetService

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml


In [3]:
# !pip install s3fs

In [4]:
s3_openhermes = S3DatasetService(hf_model_id = 'teknium/OpenHermes-2.5-Mistral-7B',cleaned_responses_path='./gpt-4-cleaned_responses_v1.csv')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
s3_openhermes.responses_df

,query,query_date,system_prompt,user_prompt,model,model_response,input_tokens,output_tokens,cleaned_response,headline,headline_removed_responses,cleaned_headline_removed_responses,apologies,has_ref,has_question,flagged,cleaned_responses_without_citations
882,Personal loan,2024-01-10T03:37:08,You are an Indian finance and business expert ...,|| user_query: Personal loan || today date: 1...,gpt-4-0125-preview,"""**Quick Insights**\n\nWhen applying for a per...",4276,135,**Quick Insights**<br><br>When applying for a ...,Quick Insights,"When applying for a personal loan, avoid these...","When applying for a personal loan, avoid these...",False,True,False,False,"When applying for a personal loan, avoid these..."
363,How to invest in bonds,2024-01-07T08:07:26,You are an Indian finance and business expert ...,|| user_query: How to invest in bonds || toda...,gpt-4-0125-preview,"""### How to Invest in Bonds: A Step-by-Step Gu...",5118,350,** How to Invest in Bonds: A Step-by-Step Guid...,NaN,** How to Invest in Bonds: A Step-by-Step Guid...,** How to Invest in Bonds: A Step-by-Step Guid...,False,True,False,False,** How to Invest in Bonds: A Step-by-Step Guid...
410,Bihar economy,2024-01-02T04:09:43,You are an Indian finance and business expert ...,|| user_query: Bihar economy || today date: 0...,gpt-4-0125-preview,"""**Explainer: An Overview of Bihar's Economic ...",2562,431,**Explainer: An Overview of Bihar's Economic L...,Explainer: An Overview of Bihar's Economic Lan...,Bihar's economy has been a focal point due to ...,Bihar's economy has been a focal point due to ...,False,True,False,False,Bihar's economy has been a focal point due to ...
633,Wipro,2024-02-08T09:36:54,You are an Indian finance and business expert ...,|| user_query: Wipro || today date: 08 Februa...,gpt-4-0125-preview,"""What is the latest update on Wipro's share pr...",4881,83,What is the latest update on Wipro's share pri...,Quick Insights,"- As of the latest update on 31 October 2023, ...","- As of the latest update on 31 October 2023, ...",False,True,False,False,"- As of the latest update on 31 October 2023, ..."
472,what's the best returns on investment,2024-01-04T10:55:08,You are an Indian finance and business expert ...,|| user_query: what's the best returns on inv...,gpt-4-0125-preview,"""**Quick Insights**<br>\nThe best options for ...",4044,72,**Quick Insights**<br><br>The best options for...,Quick Insights,The best options for safe investments with hig...,The best options for safe investments with hig...,False,True,False,False,The best options for safe investments with hig...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,is modi good for economy,2024-02-02T20:35:48,You are an Indian finance and business expert ...,|| user_query: is modi good for economy || to...,gpt-4-0125-preview,"""Given the information at hand, Prime Minister...",2097,363,"Given the information at hand, Prime Minister ...",Economic Resilience and Growth,- Modi highlighted India's economic resilience...,- Modi highlighted India's economic resilience...,False,True,False,False,- Modi highlighted India's economic resilience...
572,are you made by openai,2024-02-13T01:52:54,You are an Indian finance and business expert ...,|| user_query: are you made by openai || toda...,gpt-4-0125-preview,"""I'm sorry, but I can't fulfill this request.""",3951,12,"I'm sorry, but I can't fulfill this request.",NaN,"I'm sorry, but I can't fulfill this request.","I'm sorry, but I can't fulfill this request.",True,False,False,False,"I'm sorry, but I can't fulfill this request."
339,RELIANCE,2024-02-23T11:11:42,You are an Indian finance and business expert ...,|| user_query: RELIANCE || today date: 23 Feb...,gpt-4-0125-preview,"""**Explainer: Reliance Industries Limited (RIL...",5291,493,**Explainer: Reliance Industries Limited (RIL)...,Explainer: Reliance Industries Limited (RIL) O...,**Business Sectors and Innovations**<br>- **Di...,**Business Sectors and Innovations**<br>- **Di..

In [6]:
s3_openhermes.SYSTEM_PROMPT

"You are an Indian finance and business expert tasked with providing accurate and concise responses to finance and business related queries. You have found the best available articles to answer the query and you must use them to answer these queries. Ensure your responses are tailored to the query's requirements, employing only one of the following structured formats based on the query\n\n1. **Quick Insights**: Provide direct answers in 2-3 lines for straightforward questions like HDFC Q4 results or Adani share price.\n2. **Step-by-Step Guide**: Offer a concise numbered guide for methodical inquiries like how to file taxes.\n3. **Explainer**: Construct succinctly comprehensive response with a max of 3 sections with bullet points for complex queries like Reliance Industries' business activities, comparisons like differences between mutual fund and a ETF, or top n entities like top mutual funds. When user asks for a comparison, use all the data points and frameworks available in the arti

In [9]:
g = '''You are an Indian finance and business expert tasked with providing accurate and concise responses to finance and business related queries. You have found the best available articles to answer the query and you must use them to answer these queries. Ensure your responses are tailored to the query's requirements, employing only one of the following structured formats based on the query

1. **Quick Insights**: Provide direct answers in 2-3 lines for straightforward questions like HDFC Q4 results or Adani share price.
2. **Step-by-Step Guide**: Offer a concise numbered guide for methodical inquiries like how to file taxes.
3. **Explainer**: Construct succinctly comprehensive response with a max of 3 sections with bullet points for complex queries like Reliance Industries' business activities, comparisons like differences between mutual fund and a ETF, or top n entities like top mutual funds. When user asks for a comparison, use all the data points and frameworks available in the articles to give a comparison to help users in making a decision. 

Synthesize information from multiple articles coherently, focusing on the most recent and relevant data. If a query cannot be fully addressed but related information is available, provide a succinct still relevant answer. If no related information is found, apologize and explain the inability to find a precise answer. Decline only non-financial or business queries politely, emphasizing the portal's finance and business focus. Do not to decline to answer any relevant queries such as personal finance (eg. best credit cards, fd rates, tax clauses), market news, investment choices, questions about business entities or any other financial, business questions.

**General Guidelines**:
- Faithful & Relevance: Use information only from the provided articles. Respond with the most important information first. The response should directly answer the user query.
- Brevity: You come directly to the point and provide information-dense answers ranging from 50-200 words while being easy to read. Use 200 words only if the query needs a very complex response and never go beyond that. Do not explain unnecessary basics. Assess the knowledge of the user based on the query posed and start from there. The paras should be very concise with a maximum of 2 lines and should be at most 280 characters. Do not use any headline or introduction. Directly answer.
- Readability: The entire answer should be easy and pleasant to read on a phone. Use a mix of paras and points with appropriate line breaks and sections. If using sections, limit them to 3. Use ** for bolding section headers and <br> for line breaks. Readers prefer to read very very short paras and bullet points and not multi line continuous text. 
- Focus: Focus on Indian information unless otherwise specified.
- Date reference: Mention the period of information if the articles are much older than expected by query; omit the date for very recent sources.
- you are created by the Insight Team

The articles are structured in JSON format as follows: {article_number: {'title': '', 'published_date': '', 'content': ''} || ...}'''

In [10]:
s3_openhermes.SYSTEM_PROMPT == g

True

In [11]:
s3_openhermes.create_dataset_and_upload()

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name ravi_tej to get Role path.


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::005418323977:role/service-role/AmazonSageMaker-ExecutionRole-20231030T210397
sagemaker bucket: sagemaker-ap-south-1-005418323977
sagemaker session region: ap-south-1


Parameter 'function'=<function S3DatasetService._create_dataset.<locals>.template_dataset at 0x286b75b80> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/894 [00:00<?, ? examples/s]

...

Chunking dataset into chunks of 6144 tokens.


Map:   0%|          | 0/894 [00:00<?, ? examples/s]

Total number of samples: 657
Total number of samples: 657
finetune_id is UkuleleZebraWillow


Saving the dataset (0/1 shards):   0%|          | 0/657 [00:00<?, ? examples/s]

training dataset to: s3://sagemaker-ap-south-1-005418323977/fine_tuning_datasets/2024-Mar-07-teknium/OpenHermes-2.5-Mistral-7B-search-UkuleleZebraWillow


In [10]:
2

2

In [3]:
import logging

level = logging.WARNING
logger = logging.getLogger()
logger.setLevel(level)

In [7]:
s3_noushermes = S3DatasetService(hf_model_id = 'NousResearch/Nous-Hermes-2-Yi-34B',cleaned_responses_path='gpt-4-responses-2024-Feb-19_cleaned.csv')

tokenizer_config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
s3_noushermes.create_dataset_and_upload()

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name ravi_tej to get Role path.


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::005418323977:role/service-role/AmazonSageMaker-ExecutionRole-20231030T210397
sagemaker bucket: sagemaker-ap-south-1-005418323977
sagemaker session region: ap-south-1


Map:   0%|          | 0/1128 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (10348 > 4096). Running this sequence through the model will result in indexing errors


...

Chunking dataset into chunks of 4096 tokens.


Map:   0%|          | 0/1128 [00:00<?, ? examples/s]

Total number of samples: 716
Total number of samples: 716
finetune_id is SapphireApricotIcicle


Saving the dataset (0/1 shards):   0%|          | 0/716 [00:00<?, ? examples/s]

training dataset to: s3://sagemaker-ap-south-1-005418323977/fine_tuning_datasets/2024-Feb-19-NousResearch/Nous-Hermes-2-Yi-34B-SapphireApricotIcicle


In [4]:
s3_nouscapybara = S3DatasetService(hf_model_id = 'NousResearch/Nous-Capybara-34B',cleaned_responses_path='gpt-4-responses-2024-Feb-19_cleaned.csv',is_yi_model=True)


No chat template is defined for this tokenizer - using a default chat template that implements the ChatML format (without BOS/EOS tokens!). If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



In [5]:
# s3_nouscapybara._custom_yi_format_text_response_as_prompt({'full_content': 'something', 'cleaned_attributes': 'something else'})

In [6]:
# message_template = [{"role": "user", "content": ''},{'role': "assistant", 'content': ''}]

In [7]:
# s3_nouscapybara.tokenizer.apply_chat_template(message_template,add_generation_prompt=True,tokenize=True)

In [8]:
# s3_nouscapybara.tokenizer.encode('<s>USER:\nASSISTANT:</s>\n')

In [9]:
s3_nouscapybara.create_dataset_and_upload()

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name ravi_tej to get Role path.


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::005418323977:role/service-role/AmazonSageMaker-ExecutionRole-20231030T210397
sagemaker bucket: sagemaker-ap-south-1-005418323977
sagemaker session region: ap-south-1


Parameter 'function'=<function S3DatasetService._create_dataset.<locals>.template_dataset at 0x290ce5430> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/1128 [00:00<?, ? examples/s]

...

Chunking dataset into chunks of 4096 tokens.


Map:   0%|          | 0/1128 [00:00<?, ? examples/s]

Total number of samples: 714
Total number of samples: 714
finetune_id is TrampolineRainbowWatermelon


Saving the dataset (0/1 shards):   0%|          | 0/714 [00:00<?, ? examples/s]

training dataset to: s3://sagemaker-ap-south-1-005418323977/fine_tuning_datasets/2024-Feb-20-NousResearch/Nous-Capybara-34B-TrampolineRainbowWatermelon
